In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Application to FACT Data and Boosting


<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>


The story so far:

- Linear Discriminant Analysis (LDA) and Fisher's linear discriminant
- Principal Component Analysis (PCA)
- Feature Selection
- Supervised Learning
- Clustering

In [2]:
from ml import plots
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import ListedColormap

In [3]:
%matplotlib widget

In [4]:
pd.options.display.max_rows = 10
plots.set_plot_style()

# A Complete Example

Below we load a dataset containing data observed by the FACT telescope.

<img width="45%" src="http://www.miguelclaro.com/wp/wp-content/uploads/2013/10/FACTMilkyWayVertical-4650-net.jpg" />   

We will perform the typical steps to build and evaluate a classifier.

0. Understand where your data comes from

1. Preprocessing
    * Drop Constant Values,
    * Handle Missing Data 
    * Feature Generation

2. Splitting
    
    * Split your data into training and evaluation sets
    
3. Training 
    
    * Train your classifier of choice.
    
4. Evaluation
    
    * Evaluate the performance on the test data set.
    * If not good enough, go back to step 1 
    
5. Physics
    
    * Check whether your data support your hypothesis
    

## 1. Get to know your data

Cherenkov telescopes record short flashes of light produced by very high energy cosmic rays and photons hitting earths atmosphere.

![](https://www.cta-observatory.org/wp-content/uploads/2016/05/cta47.png)

In [5]:
%%HTML
<!-- https://nextcloud.e5.physik.tu-dortmund.de/index.php/s/e7yb2mifGDeyDBN/download -->
<video width="100%" controls>
  <source src="./resources/fact_events.mp4" type="video/mp4">
</video>

We will use machine learning for two tasks in this example. 

 * Train a classifier to distinguish events induced by gamma rays form events induced by cosmic rays
 * Train a regressor to estimate the energy of the incoming primary particle.

## 2. Preprocess data

A lot of preprocessing has already happened at this point.

* Calibration of Raw Data
* Data Reduction from voltage timeseries per pixel to #photons and mean time for each pixel
* Calculation of image features


Load data and remove unwanted columns and store the true labels separately.

In [6]:
import pandas as pd
from fact.io import read_h5py

In [7]:
gammas = read_h5py('./resources/sample_diffuse_gammas.h5', key='events')
gammas.head()

,arrival_time_mean,arrival_time_pedestal_kurtosis,arrival_time_pedestal_max,arrival_time_pedestal_mean,arrival_time_pedestal_median,arrival_time_pedestal_min,arrival_time_pedestal_p25,arrival_time_pedestal_p75,arrival_time_pedestal_skewness,arrival_time_pedestal_variance,...,time_gradient_slope_long,time_gradient_slope_long_err,time_gradient_slope_trans,time_gradient_slope_trans_err,time_gradient_sse_long,time_gradient_sse_trans,timespread,timespread_weighted,trigger_type,width
0,50.064510,-1.109841,96.251409,50.056320,48.541463,5.000000,30.315271,70.190476,0.122054,548.930450,...,-0.028107,0.072398,0.184584,0.033308,3.345592,0.312691,0.838293,0.665250,4,4.125594
1,49.971349,-1.081534,99.000000,49.938162,49.299742,5.000000,30.720019,69.295873,0.100216,539.360854,...,-0.005743,0.039438,0.021480,0.066726,3.880951,3.818274,0.746172,0.802401,4,5.008837
2,50.364374,-1.137693,99.000000,50.406649,48.931727,5.000000,30.915493,70.418605,0.144873,558.159445,...,-0.025479,0.034733,-0.046940,0.051830,6.143687,5.922080,0.857385,0.695980,4,5.713048
3,49.754275,-1.088434,98.000000,49.789320,47.707045,6.176471,31.011719,69.293104,0.153315,539.759137,...,0.065316,0.047992,0.144199,0.099693,3.735585,3.588492,0.954412,0.912319,4,4.236660
4,49.341319,-1.133003,98.207941,49.314529,47.447368,5.000000,29.168577,69.760411,0.154019,560.212447,...,-0.004945,0.109799,0.277320,0.135065,124.030428,84.484591,3.358279,2.771534,4,6.694258


Now delete all simulated values which can not be observed during measurement in the physical world. We know which columns to remove because they have a special prefix.

In [8]:
forbidden_columns = 'ceres_|mc_|corsika_|run_|source_position_|pointing_|aux_|event_num|incident_angle|.*pedestal|fluct_|ped_'
gammas = gammas.filter(regex=f'^(?!{forbidden_columns}).*$')

len(gammas.columns)

63

Check the data types of the columns. We can select non-numeric types and encode them. But in this case we might as well drop them as the attribute is not important.

In [9]:
c = gammas.select_dtypes(exclude=['number']).columns
print(c)

gammas = gammas.drop(c, axis='columns')

Index(['runtype'], dtype='object')


We can spot the columns with constant values by looking at the standard deviation.

In [10]:
desc = gammas.describe()
desc

,arrival_time_mean,arrival_time_shower_kurtosis,arrival_time_shower_max,arrival_time_shower_mean,arrival_time_shower_min,arrival_time_shower_skewness,arrival_time_shower_variance,cog_x,cog_y,concentration_cog,...,time_gradient_slope_long,time_gradient_slope_long_err,time_gradient_slope_trans,time_gradient_slope_trans_err,time_gradient_sse_long,time_gradient_sse_trans,timespread,timespread_weighted,trigger_type,width
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000
mean,49.740845,1.669750,54.761202,51.848245,49.116457,0.084997,3.003343,1.319683,-14.999594,0.341442,...,0.001122,0.036415,0.001657,0.071118,32.358447,66.361993,1.443372,1.273125,4.0,6.504231
std,0.765677,3.260473,4.513202,2.979274,2.996825,1.185409,5.509836,84.890029,87.004145,0.175294,...,0.068504,0.034970,0.100130,0.061352,56.183769,344.951531,0.817965,0.740511,0.0,1.755483
min,47.316001,-3.308731,41.170213,39.042000,35.359862,-4.533000,0.019868,-175.398979,-178.151067,-0.030815,...,-0.491409,0.000947,-0.790551,0.002228,0.028250,0.027688,0.126072,0.131271,4.0,2.064943
25%,49.241067,-0.469649,51.888632,50.060331,47.572560,-0.593028,0.955858,-65.547247,-86.870373,0.202670,...,-0.033493,0.014003,-0.041412,0.033228,6.641634,7.048601,0.928098,0.828031,4.0,5.223795
50%,49.706151,0.717916,54.027215,51.734115,49.356218,0.124951,1.700814,1.397124,-21.136853,0.300876,...,0.001012,0.026125,0.002579,0.053872,15.690777,17.889471,1.243882,1.093557,4.0,6.296922
75%,50.178076,2.785657,56.886122,53.624683,51.000000,0.777732,3.255480,68.424271,53.570215,0.467830,...,0.034840,0.046479,0.043885,0.088357,37.561407,45.605035,1.728471,1.499146,4.0,7.616817
max,58.051201,26.580277,91.434777,68.149130,60.483146,4.392427,200.727871,179.151919,173.541225,0.839026,...,1.187848,0.387313,0.853158,1.387731,1944.673229,13904.011576,13.357574,13.135994,4.0,28.411963


In [11]:
c = desc.columns[desc.loc['std'] == 0]
print(c)
gammas = gammas.drop(c, axis='columns')

Index(['num_pixel', 'roi', 'trigger_type'], dtype='object')


Check for missing data. (Just delete it in this case)

In [12]:
gammas = gammas.dropna()

So far we only loaded simulated gamma-ray showers. Now we do the same for the cosmic ray events. We create a method to perform all preprocessing in one step. We need this several times.

In [13]:
def preprocess(df):
    df = df.filter(regex=f'^(?!{forbidden_columns}).*$')
    c = df.select_dtypes(exclude=['number']).columns
    df = df.drop(c, axis='columns')
    desc = df.describe()
    c = desc.columns[desc.loc['std'] == 0]
    df = df.drop(c, axis='columns')
    df = df.dropna()
    return df

In [14]:
protons = read_h5py('./resources/sample_proton.h5', key='events')
protons = preprocess(protons)

Now we can perform feature generation. We use our expert knowledge or intuition to create a new feature by combining existing columns into a new variable.

In [15]:
def feature_generation(df):
    df['awesome_feature'] =  df['size'] * (df['width'] / df['length'])
    return df

gammas = feature_generation(gammas)
protons = feature_generation(protons)

gammas['awesome_feature']

0       18.606696
1       21.858272
2       39.078776
3       13.730881
4       29.674442
          ...    
9995    20.295848
9996    19.856191
9997    36.360764
9998    16.875930
9999    75.253902
Name: awesome_feature, Length: 10000, dtype: float64

A quick look at the data so far

In [22]:
bins = np.linspace(0, 70, 100)
# bins = np.logspace(0, 5, 100)
# bins = 100

col = 'length'

plt.figure()
plt.hist(gammas[col], bins=bins, histtype='step', lw=2, label='Gammas')
plt.hist(protons[col], bins=bins, histtype='step', lw=2, label='Protons')
# plt.xscale('log')
plt.legend()
None

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

At this point we combine the two datasets into one big matrix and build a label vector $y$

In [23]:
X = pd.concat([gammas, protons])
y = np.concatenate([np.ones(len(gammas)), np.zeros(len(protons))])

## 3. Split Data

Now we can split the data into test and training sets. Scikit-Learn provides some neat methods to do just that.

In [24]:
from sklearn.model_selection import train_test_split
X_test, X_train, y_test, y_train = train_test_split(X, y)

## 4. Train the classifier

Now we can train any classifier we want on the prepared data.

In [27]:
from sklearn.tree import DecisionTreeClassifier

rf = DecisionTreeClassifier(max_depth=15, criterion='entropy')
rf.fit(X_train, y_train)

y_prediction = rf.predict(X_test)
y_prediction_proba = rf.predict_proba(X_test)

## 5. Evaluation 

Check accuracy of the models and other metrics 

In [28]:
importance = pd.Series(rf.feature_importances_, index=gammas.columns)


plt.figure()
importance.sort_values().tail(20).plot.barh()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [29]:
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score

acc = accuracy_score(y_test, y_prediction)
auc = roc_auc_score(y_test, y_prediction_proba[:, 1])
fpr, tpr, thresholds = roc_curve(y_test, y_prediction_proba[:, 1])

In [30]:

plt.figure()
plt.scatter(fpr, tpr, c=thresholds, vmax=1)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.gca().set_aspect(1)
plt.plot(fpr, tpr, '--', color='gray', alpha=0.5)
plt.text(0.5, 0.5, f'AuC ROC: {auc:0.03f} \nAccuracy: {acc:0.03f}')
plt.colorbar()
None

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Perform steps 3, 4, and 5 in one step using cross validation

In [31]:
from sklearn.model_selection import cross_validate

rf = DecisionTreeClassifier(max_depth=12, criterion='entropy')

scoring = {'acc': 'accuracy',
           'auc': 'roc_auc',
           'recall': 'recall'}

results = cross_validate(rf, X, y, cv=5, scoring=scoring, return_train_score=True)
results

{'fit_time': array([1.33849788, 1.28981209, 1.30354881, 1.29822016, 1.29255605]),
 'score_time': array([0.01068687, 0.00917387, 0.00890326, 0.00941944, 0.00946593]),
 'test_acc': array([0.66775, 0.6745 , 0.639  , 0.67   , 0.66325]),
 'train_acc': array([0.801    , 0.7945   , 0.797125 , 0.7893125, 0.7855   ]),
 'test_auc': array([0.710017  , 0.7138055 , 0.68766188, 0.70907462, 0.7115545 ]),
 'train_auc': array([0.89342185, 0.89314156, 0.89691017, 0.88981106, 0.88098145]),
 'test_recall': array([0.76  , 0.7635, 0.6745, 0.7175, 0.758 ]),
 'train_recall': array([0.890625, 0.891125, 0.842   , 0.84125 , 0.873   ])}

In [32]:
auc = results['test_auc']
recall = results['test_recall']
acc = results['test_acc']

print(f'Area under RoC curve: {auc.mean():0.04f} ± {auc.std():0.04f}')
print(f'Accuracy:             {acc.mean():0.04f} ± {acc.std():0.04f}')
print(f'Recall:               {recall.mean():0.04f} ± {recall.std():0.04f}')

Area under RoC curve: 0.7064 ± 0.0095
Accuracy:             0.6629 ± 0.0125
Recall:               0.7347 ± 0.0344


## 6. Physics

Now we could test our model and our hypothesis on real observed data. This part of the analysis is the most time 
consuming in general. It also requires more data than than this notebook can handle. 
After careful analysis one can produce an image of the gamma-ray sky

<img width="60%" src="https://www.mpi-hd.mpg.de/hfm/HESS/hgps/figures/HESS_J1813m126.png">

## Improving Classification


### Boosting and AdaBoost

Similar to the idea of combining many classifiers through bagging (like we did for the RandomForests) we now 
train many estimators in a sequential manner. In each iteration the data gets modified slightly using weights $w$
for each sample in the training data. In the first iteration the weights are all set to $w=1$

In each successive iteration the weights are updated. The samples that were incorrectly classified in the previous 
iteration get a higher weight. The weights for correctly classified samples get decreases. 
In other words: We increase the influence/importance of samples that are difficult to classify.

Predictions are performed by taking a weighted average of the single predictors.

The popular AdaBoost algorithms takes this a step further by optimizing the weight of each separate classifier 
in the ensemble.
The AdaBoost ensemble combines many learners in an iterative way. The learner at iteration $m$ is:

$$
 F_{m}(x)=F_{m-1}(x)+\gamma _{m}h_{m}(x)
$$

The choice of $F_0$ is problem specific.

Each weak learner produces a prediction $h(x_{m})$ for each sample in the training set. At each iteration $m$ a 
weak learner is fitted and assigned a coefficient $\gamma_{m}$ which is found by minimizing:

$$
\gamma_m = {\underset {\gamma }{\arg \min }} \sum_{i}^{N}E\bigl(F_{m-1}(x_{i})+\gamma h(x_{i})\bigr)
$$

where $E(F)$ is some error function and $x_i$ is the reweighted data sample.

In general this method can work with any classifying method. Traditionally it is being used with very small 
decision trees. 
The weights get used to select the split points during the minimization of the loss function in each node

$$
 \underset{(X, s) \in \, \mathbf{X} \times {S}}{\arg \max} IG(X,Y) =   \underset{(X, s) \in \, \mathbf{X} \times {S}}{\arg \max} ( H(Y) - H(Y |\, X) ).
$$

Below we try AdaBoost on the FACT data.


In [33]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=2),
    n_estimators=100,
    learning_rate=0.5,
)
ada.fit(X_train, y_train)

y_prediction = ada.predict(X_test)
y_prediction_proba = ada.predict_proba(X_test)

In [34]:
scores = np.array(list(ada.staged_score(X_test, y_test)))

plt.figure()
plt.plot(scores, '.')
plt.ylabel('Accuracy')
plt.xlabel('Iteration')
None

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
acc = accuracy_score(y_test, y_prediction)
auc = roc_auc_score(y_test, y_prediction_proba[:, 1])
fpr, tpr, thresholds = roc_curve(y_test, y_prediction_proba[:, 1])

plt.figure()
plt.scatter(fpr, tpr, c=thresholds)
plt.plot(fpr, tpr, '--', color='gray', alpha=0.5)
plt.text(0.5, 0.5, f'AuC ROC: {auc:0.03f} \nAccuracy: {acc:0.03f}')
None

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Gradient Boosting 

Very similar to AdaBoost. Only this time we change the target label we train the classifiers for.

Formulate the general problem as follows (See Wikipedia):

Starts with a constant function $F_{0}(x)$ and some differentiable loss function $L$ and incrementally expands it in a greedy fashion:

$$
F_{0}(x)={\underset {\gamma }{\arg \min }}{\sum _{i=1}^{n}{L(y_{i},\gamma )}}
$$

$$
F_{m}(x)=F_{m-1}(x)+{\underset {h_{m}\in {\mathcal {H}}}{\operatorname {arg\,min} }}\left[{\sum _{i=1}^{n}{L(y_{i},F_{m-1}(x_{i})+h_{m}(x_{i}))}}\right]
$$

Finding the best $ h_{m}\in {\mathcal {H}}$ is computationally speaking impossible.
If we could find the perfect $h$ however, we know that 

$$
F_{m+1}(x_i)=F_{m}(x_i)+h(x_i)=y_i
$$

or, equivalently, 

$$
   h(x_i)= y_i - F_{m}(x_i)
$$

Note that for the mean squared error loss $\frac{1}{2}(y_i - F(x_i))^2$ this is equivalent to the negative 
gradient with respect to $F_i$.

For a general loss function we fit $h_{m}(x)$ to the residuals, or negative gradients 
$$
 r_{i, m}=-\left[{\frac {\partial L(y_{i},F(x_{i}))}{\partial F(x_{i})}}\right]_{F(x)=F_{m-1}(x)}\quad {\mbox{for }}i=1,\ldots ,n.
$$



Below we try it on FACT data again.


In [36]:
from sklearn.ensemble import GradientBoostingClassifier

grb = GradientBoostingClassifier(
    verbose=True,
    n_estimators=100,
)
grb.fit(X_train, y_train)

y_prediction = grb.predict(X_test)
y_prediction_proba = grb.predict_proba(X_test)

      Iter       Train Loss   Remaining Time 
         1           1.3546            7.15s
         2           1.3273            6.90s
         3           1.3057            6.79s
         4           1.2880            6.71s
         5           1.2722            6.60s
         6           1.2568            6.52s
         7           1.2448            6.45s
         8           1.2281            6.40s
         9           1.2151            6.33s
        10           1.2059            6.28s
        20           1.1298            5.53s
        30           1.0871            5.14s
        40           1.0543            4.44s
        50           1.0270            3.64s
        60           1.0048            2.88s
        70           0.9873            2.14s
        80           0.9741            1.43s
        90           0.9566            0.71s
       100           0.9416            0.00s


In [37]:
l = [accuracy_score(y_pred, y_test) for y_pred in grb.staged_predict(X_test)]

plt.figure()
plt.plot(range(len(l)), l, '.')
plt.ylabel('Accuracy')
plt.xlabel('Iteration')
None

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
acc = accuracy_score(y_test, y_prediction)
auc = roc_auc_score(y_test, y_prediction_proba[:, 1])
fpr, tpr, thresholds = roc_curve(y_test, y_prediction_proba[:, 1])

plt.figure()
plt.scatter(fpr, tpr, c=thresholds)
plt.plot(fpr, tpr, '--', color='gray', alpha=0.5)
plt.text(0.5, 0.5, f'AuC ROC: {auc:0.03f} \nAccuracy: {acc:0.03f}')
None

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

More on gradient descent algorithms can be found in the Neural Network notebook.

Let's now test our all time favorite classifier. 

In [39]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=150,  max_depth=18, criterion='entropy')
rf.fit(X_train, y_train)

y_prediction = rf.predict(X_test)
y_prediction_proba = rf.predict_proba(X_test)

In [40]:
acc = accuracy_score(y_test, y_prediction)
auc = roc_auc_score(y_test, y_prediction_proba[:, 1])
fpr, tpr, thresholds = roc_curve(y_test, y_prediction_proba[:, 1])

plt.figure()
plt.scatter(fpr, tpr, c=thresholds)
plt.plot(fpr, tpr, '--', color='gray', alpha=0.5)
plt.text(0.5, 0.5, f'AuC ROC: {auc:0.03f} \nAccuracy: {acc:0.03f}')
None

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …